In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-sln3zo9l
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-sln3zo9l
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=bc56d7e27aa15c6c0df7efd02ac0e23b66f1597b936569386cbf34b6a176a593
  Stored in directory: /tmp/pip-ephem-wheel-cache-bp7ghrts/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
%%cuda --name coloring.h
#pragma once

#include "curand_kernel.h"
#include "/content/drive/MyDrive/graph/common.h"
#include "/content/drive/MyDrive/graph/graph.h"


struct Coloring {
	bool		uncoloredNodes;
	uint		numOfColors;
	uint	*	coloring;   // each element denotes a color
};


Coloring* MISColoring(GraphStruct*);
void printColoring (Coloring*, GraphStruct*, bool);
__global__ void MIScolorer (Coloring*, GraphStruct*, uint*) ;
__global__ void init(uint seed, curandState_t*, uint*, uint);
__global__ void findIS (Coloring*, GraphStruct*, uint*);
__global__ void print_d(GraphStruct*, bool);

'File written in /content/src/coloring.h'

In [58]:
%%cuda --name MIS.cu

#include "/content/src/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"

#define THREADxBLOCK 128

using namespace std;

Coloring* MISColoring(GraphStruct *str){
  Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

	// cudaMalloc for arrays of struct Coloring
	CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	// allocate space on the GPU for the random states
	curandState_t* states;
	uint* weigths;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;
	init <<< blocks, threads >>> (seed, states, weigths, n);

	// start coloring (dyn. parall.)
	MIScolorer <<< 1, 1 >>> (col, str, weigths);

  cudaFree(states);
	cudaFree(weigths);
  return col;
}

__global__ void findIS (Coloring* col, GraphStruct *str, uint* weights, bool *currentIS, bool *X) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
	for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
		if (!X[neighID] && !col->coloring[neighID] &&
				((weights[idx] < weights[neighID]) ||
				((weights[idx] == weights[neighID]) && idx < neighID))) {
			candidate = false;
		}
	}
	if (candidate) {
		currentIS[idx] = true;
    X[idx] = true;
    for (uint j = 0; j < deg; j++) {
        uint neighID = str->neighs[offset + j];
        X[neighID] = true;
    }
	}
}

__global__ void init (uint seed, curandState_t* states, uint* numbers, uint n) {
	uint idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx > n)
			return;
	curand_init(seed, idx, 0, &states[idx]);
	numbers[idx] = curand(&states[idx])%n*n;
}

__global__ void checkX(GraphStruct *str, bool* X, bool* result) {
		uint idx = threadIdx.x + blockDim.x * blockIdx.x;

    if(X[idx] == false)
      &result = true;
}

__global__ void colorMIS(Coloring* col, GraphStruct *str, bool *currentIS){
  uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

  if(currentIS[idx])
      col->coloring[idx] = col->numOfColors;
}

__global__ void MIScolorer (Coloring* col, GraphStruct *str, uint* weights) {
	dim3 threads (THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

  bool* currentIS;
	uint n = str->nodeSize;
  cudaMalloc((void**) &currentIS, n * sizeof(bool));
  memset(currentIS, false, n);

  bool* X;
  cudaMalloc((void**) &X, n * sizeof(bool));
  memset(X, false, n);

	// loop on ISs covering the graph
	col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
		col->numOfColors++;

		bool result;
		checkX <<< blocks, threads >>> (str, X, &result);

    while(result){
        findIS <<< blocks, threads >>> (col, str, weights, currentIS, X);
		    //cudaDeviceSynchronize();
				checkX <<< blocks, threads >>> (str, X, &result);
    }

    colorMIS <<< blocks, threads >>> (col, str, currentIS);
	}
}

/**
 * Print the graph (verbose = 1 for "verbose print")
 * @param verbose print the complete graph
 */


'File written in /content/src/MIS.cu'

In [26]:
%%cuda --name test_MIS.cu

#include "/content/src/coloring.h"

using namespace std;

int main(void) {
	unsigned int n = 100;		 // number of nodes for random graphs
	float prob = .5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

  srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = MISColoring(str);
	cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  //Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	//printColoring(col, str, 1);

	return EXIT_SUCCESS;
}

'File written in /content/src/test_MIS.cu'

In [59]:
!nvcc -dc src/test_MIS.cu /content/src/MIS.cu /content/drive/MyDrive/graph/graph.cpp /content/drive/MyDrive/graph/graph_d.cu
!nvcc test_MIS.o MIS.o graph.o graph_d.o -o test_MIS
!./test_MIS

/content/src/MIS.cu(80): error: expression must be a modifiable lvalue

1 error detected in the compilation of "/content/src/MIS.cu".
nvlink fatal   : Could not open input file 'MIS.o'
/bin/bash: line 1: ./test_MIS: No such file or directory
